# SkyNet - A Language Model for Games
Developed by DevArtech (Adam Haile)

Skyjo is a card game which can be played by 2 or more players. The goal of the game is to end the game with the lowest score. The game ends when any player gets over 100 points at the end of any round.
The game is setup as follows:
A deck consists of: 
- 5 cards with a value of -2
- 10 cards with a value of -1
- 15 cards with a value of 0
- Values 1-12, 10 cards of each

To start the game, each player is dealt a total of 12 cards. These cards are not to be seen, and are generally laid in a 4x3 grid in front of the player.
On the first round, each player flips over 2 cards of their choice to start. For the sake of this game, 2 cards are chosen at random. The player with the highest total starts first.
Players then perform the following actions in the given order:
- Choose a deck to draw from. Players can either draw from the deck (which is unknown), or the last discard card (if it exists, this card is known).
- Choose to keep drawn card, or discard it. (Generally, if drawn from the discard, this card is always kept)
- Chose a card on their grid to perform the following action with:
   - If discarded, chose an unknown card of their hand to flip
   - If not discard, chose any card to replace with their drawn card
Once a player has performed this last action, their turn is done and play passes to the next player.
A round ends once any player has revealed all of their cards. When this happens, all players asides from this player have one more turn before the round ends.

When a round ends, players total the values of all of their cards, and add it to their game total score.
If any one player has a lower score than the player who revealed all their cards first, the player who "went out" first has to double their round's score.
This repeats till any player hits 100 points or more at the end of a round, and the player with the lowest score at this time wins.

There is one special rule which is optional to play with, but is included for this model.
If, at any point, a player reveals 3 of the same card in one column, this column is then cleared and removed for the rest of the round. Cards can be of any value though, so while clearing a column of 12s is highly beneficial, clearing a column of -2s is highly detrimental.
The cleared cards are then added to the top of the discard.


The purpose of this model is to play a game of Skyjo. The model is trained by playing against 5 different algorithms, each of which is given different instructions on what to do for a given turn.
- RandomBot: Always choses actions at random. (Random draw, random keep, random placement)
- MiddleBot: Performs actions which are always low (If discard is below 7, take it. If card drawn is below 7, keep it. If drawn card is lower than any visible, replace it, if not, place it in the next unknown index)
- SpeedBot: Performs actions as quickly as possible (Same logic as MiddleBot asides from placement. Unless drawn card can create highly benefical point decreases, given card on board is > 4 and drawn card < given card - swap them, it will place in the next unknown spot.)
- SmartBot: General logic to perform as human as possible (If discard is lower than any given card on the board or low, take it. If drawn card is lower than any card on the board or is medium, keep it. Check all cards on the board and find the largest beneficial point difference to swap with.)
- TripleBot: Performs actions which will create columns of the same card. (If discard is medium or card is on the board, take it. If drawn card is on the board medium, keep it. If card can be placed at a given index and create a clearable column, place it there, otherwise use same logic as Speed for placement.)

These algorithms will be chosen at random to play against the model, with the hope that a diversified set of algorithms will create a model capable to combatting many types of gameplay

This main notebook is meant to be used for the execution of the game, as well as creating and training a SkyNet model.

First, we import packages to create the game and other helper methods

In [1]:
import random
import time
import pygame
import game
import bots
import tqdm
import tensorflow as tf
import numpy as np
from skynet import Skynet

pygame 2.1.0 (SDL 2.0.16, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 

Next, we create the game enviornment.

In [ ]:
g = game.Skyjo()

Next, we define the type of game. For sake of ease, games can be 2 player at most. Here, we can set whether or not we want two bots to play each other, or a bot and a player. Also included is a helper method to create the bot objects for the game.

In [ ]:
def randomBot():
    """
    Sets players at random depending on if one player or two player
    """
    rand = bots.Random()
    mid = bots.Middle()
    speed = bots.Speed()
    smart = bots.Smart()
    triple = bots.Triple()
    if two_bot:
        for i in range(2):
            bot = random.randint(0, 4)
            if bot == 0:
                if i == 0:
                    g.player1 = bots.Random(4, 5, 6, 7, 0, 1, 2, 3)
                    print("Bot is RandomBot")
                else:
                    g.player2 = rand
                    print("Bot is RandomBot")
            elif bot == 1:
                if i == 0:
                    g.player1 = bots.Middle(4, 5, 6, 7, 0, 1, 2, 3)
                    print("Bot is MiddleBot")
                else:
                    g.player2 = mid
                    print("Bot is MiddleBot")
            elif bot == 2:
                if i == 0:
                    g.player1 = bots.Speed(4, 5, 6, 7, 0, 1, 2, 3)
                    print("Bot is SpeedBot")
                else:
                    g.player2 = speed
                    print("Bot is SpeedBot")
            elif bot == 3:
                if i == 0:
                    g.player1 = bots.Smart(4, 5, 6, 7, 0, 1, 2, 3)
                    print("Bot is SmartBot")
                else:
                    g.player2 = smart
                    print("Bot is SmartBot")
            else:
                if i == 0:
                    g.player1 = bots.Triple(4, 5, 6, 7, 0, 1, 2, 3)
                    print("Bot is TripleBot")
                else:
                    g.player2 = triple
                    print("Bot is TripleBot")
    else:
        bot = random.randint(0, 4)
        if bot == 0:
            g.player2 = rand
            print("Bot is RandomBot")
        elif bot == 1:
            g.player2 = mid
            print("Bot is MiddleBot")
        elif bot == 2:
            g.player2 = speed
            print("Bot is SpeedBot")
        elif bot == 3:
            g.player2 = smart
            print("Bot is SmartBot")
        else:
            g.player2 = triple
            print("Bot is TripleBot")


# If playing two bots against each other, set two_bot to true, else set to False and specify the type of bot for Player 2 in the initialize_bots method
two_bot = False
if two_bot:
    randomBot()
else:
    g.initialize_bots(bots.Smart())
g.new_round()

Next, we set up the PyGame enviornment

In [ ]:
running = True

if running:
    pygame.init()
    bounds_x = 1024
    bounds_y = 768
    window = pygame.display.set_mode([bounds_x, bounds_y])
    pygame.display.set_caption("Skyjo")

Now, we create the card objects for display. This is to allow easier instantiation in the game

In [ ]:
card_width = 100
card_height = 150

DEFAULT = (150, 100, 200)
DEEP_BLUE = (100, 50, 255)
LIGHT_BLUE = (150, 150, 255)
GREEN = (125, 255, 125)
YELLOW = (255, 255, 100)
RED = (255, 100, 100)
font = pygame.font.Font(None, 75)
skyfont = pygame.font.Font(None, 40)
smallfont = pygame.font.Font(None, 22)

class Card():
    def __init__(self, value, hidden_value=12, position=0):
        super().__init__()
        self.image = pygame.Surface((card_width, card_height))
        self.rect = self.image.get_rect()
        self.value = value
        self.position = position
        self.hidden_value = hidden_value

        if self.value is None:
            self.image.fill(DEFAULT)
        else:
            if self.value < 0:
                self.image.fill(DEEP_BLUE)
            if self.value == 0:
                self.image.fill(LIGHT_BLUE)
            if self.value > 0 and self.value < 5:
                self.image.fill(GREEN)
            if self.value >= 5 and self.value < 9:
                self.image.fill(YELLOW)
            if self.value >= 9:
                self.image.fill(RED)

        self.color = (0, 0, 0)

        width = card_width
        height = card_height
        pygame.draw.rect(self.image, self.color, [0, 0, width, height], 3)

        if self.value is None:
            text = skyfont.render("SKYJO", True, self.color)
            self.image.blit(
                text, [7.5, card_height/2 - 15, text.get_rect().width, text.get_rect().height])
        else:
            text = font.render(str(value).capitalize(), True, self.color)
            if len(str(value)) == 2:
                self.image.blit(text, [card_width/2 - 27, card_height /
                                2 - 25, text.get_rect().width, text.get_rect().height])
            if len(str(value)) == 1:
                self.image.blit(text, [card_width/2 - 15, card_height /
                                2 - 25, text.get_rect().width, text.get_rect().height])

    def move_to(self, x, y):
        self.rect.x = (x - card_width / 2)
        self.rect.y = (y - card_height / 2)

    def draw(self, surface):
        surface.blit(self.image, self.rect)

    def get_rect(self):
        return self.rect

In [ ]:
top_discard = Card(None)
card_drawn = None

while running:
    player_one_game_cards = []
    player_two_game_cards = []
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    window.fill((95, 141, 186))
    i = 0

    p1 = None
    p2 = None
    if g.get_player_score(1) < g.get_player_score(2):
        p1 = GREEN
        p2 = RED
    elif g.get_player_score(2) < g.get_player_score(1):
        p1 = RED
        p2 = GREEN
    else:
        p1 = YELLOW
        p2 = YELLOW

    text = skyfont.render("Current Player: " +
                            str(g.current_player), True, (255, 255, 255))
    window.blit(text, [bounds_x/2 - 125, bounds_y - 75,
                text.get_rect().width, text.get_rect().height])

    text = smallfont.render(
        "Total Player Score: " + str(g.player_one), True, (255, 255, 255))
    window.blit(text, [bounds_x/20, card_height/2 - 35,
                text.get_rect().width, text.get_rect().height])

    text = skyfont.render(
        "Player Score: " + str(g.get_player_score(1)), True, p1)
    window.blit(text, [bounds_x/20, card_height/2 - 15,
                text.get_rect().width, text.get_rect().height])

    text = smallfont.render(
        "Total Player Score: " + str(g.player_two), True, (255, 255, 255))
    window.blit(text, [bounds_x/20 + 500, card_height/2 -
                35, text.get_rect().width, text.get_rect().height])

    text = skyfont.render(
        "Player Score: " + str(g.get_player_score(2)), True, p2)
    window.blit(text, [bounds_x/20 + 500, card_height/2 -
                15, text.get_rect().width, text.get_rect().height])

    for y in range(3):
        for x in range(4):
            if g.player_one_total[i] is not None:
                new_card = Card(g.player_one_visible[i], g.player_one_total[i], i)
                new_card.move_to((bounds_x / 10) + (x * 105), (bounds_y / 4) + (y * 200) - 25)
                new_card.draw(window)
                player_one_game_cards.append(new_card)

            if g.player_two_total[i] is not None:
                new_card = Card(g.player_two_visible[i], g.player_one_total[i], i)
                new_card.move_to((bounds_x / 10) + (x * 105) + 500, (bounds_y / 4) + (y * 200) - 25)
                new_card.draw(window)
                player_two_game_cards.append(new_card)

            i += 1

    if len(g.discarded_cards) > 0:
        top_discard = Card(g.discarded_cards[0])
        top_discard.move_to((bounds_x / 10) + 650, bounds_y - 35)
        top_discard.draw(window)

    if g.drawn_card != -3:
        card_drawn = Card(g.drawn_card)
        card_drawn.move_to((bounds_x / 10) + 755, bounds_y - 35)
        card_drawn.draw(window)

    deck = Card(None)
    deck.move_to((bounds_x / 10) + 860, bounds_y - 35)
    deck.draw(window)

    if event.type == pygame.MOUSEBUTTONDOWN:
        mouse_pos = pygame.mouse.get_pos()

        if g.current_player == 1:
            for card in player_one_game_cards:
                if card.get_rect().collidepoint(mouse_pos):
                    if (g.swap_from_deck is True or g.swap_from_discard is True) and g.keep_drawn:
                        g.switch_current()
                        g.discard(g.player_one_total[card.position])
                        g.swap_card(1, card.position, g.drawn_card)
                        g.drawn_card = -3
                        g.swap_from_deck = False
                        g.swap_from_discard = False
                        g.keep_drawn = True
                        if g.last_swap:
                            g.round_over = True
                    elif (g.swap_from_deck is True or g.swap_from_discard is True) and not g.keep_drawn:
                        if card.value != card.hidden_value:
                            g.switch_current()
                            g.swap_card(1, card.position, card.hidden_value)
                            g.drawn_card = -3
                            g.swap_from_deck = False
                            g.swap_from_discard = False
                            g.keep_drawn = True
                            if g.last_swap:
                                g.round_over = True

        # Comment in if you want to players to play each other instead of player/bot or bot/bot
        # if g.current_player == 2:
        #     for card in player_two_game_cards:
        #         if card.get_rect().collidepoint(mouse_pos):
        #             if (g.swap_from_deck is True or g.swap_from_discard is True) and g.keep_drawn:
        #                 g.switch_current()
        #                 g.discard(g.player_two_total[card.position])
        #                 g.swap_card(2, card.position, g.drawn_card)
        #                 g.drawn_card = -3
        #                 g.swap_from_deck = False
        #                 g.swap_from_discard = False
        #                 if g.last_swap:
        #                     g.round_over = True
        #             elif (g.swap_from_deck is True or g.swap_from_discard is True) and not g.keep_drawn:
        #                 if card.value != card.hidden_value:
        #                     g.switch_current()
        #                     g.swap_card(1, card.position, card.hidden_value)
        #                     g.drawn_card = -3
        #                     g.swap_from_deck = False
        #                     g.swap_from_discard = False
        #                     g.keep_drawn = True
        #                     if g.last_swap:
        #                         g.round_over = True

        if card_drawn is not None and card_drawn.get_rect().collidepoint(mouse_pos) and g.keep_drawn and g.drawn_card != -3:
            g.dump_drawn(0)
            g.drawn_card = -3
            card_drawn = None

        if deck.get_rect().collidepoint(mouse_pos) and g.swap_from_discard is False and g.drawn_card == -3 and g.keep_drawn:
            g.swap_from_deck = True
            g.drawn_card = g.draw_deck()

        if top_discard.get_rect().collidepoint(mouse_pos) and g.swap_from_deck is False and g.drawn_card == -3:
            g.swap_from_discard = True
            g.drawn_card = g.draw_discard()

    if g.is_round_over():
        g.last_swap = True

    if g.round_over:
        result = g.tally_round_score(running)
        if result == 1:
            print("Player 1 Wins!")
            running = False
        elif result == 2:
            print("Player 2 Wins!")
            running = False
        elif result == 3:
            print("Players tied!")
            running = False

    if g.current_player == 1 and g.player1 is not None and running:
        g.card_choice(g.player1.get_card_choice(g.get_round_state()))
        g.dump_drawn(g.player1.get_keep_choice(g.get_round_state()))
        g.player_one_placement_choice(g.player1.get_placement_choice(g.get_round_state()))
    if g.current_player == 2 and g.player2 is not None and running:
        g.card_choice(g.player2.get_card_choice(g.get_round_state()))
        g.dump_drawn(g.player2.get_keep_choice(g.get_round_state()))
        g.player_two_placement_choice(g.player2.get_placement_choice(g.get_round_state()))

    pygame.display.flip()
    if two_bot:
        time.sleep(0.25)
        
pygame.quit()

Player 1 Wins!
